# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# 1. Read the CSV into a dataframe
df = pd.read_csv('tic-tac-toe.csv')

# 2. Inspect the dataset
print(df.head())
print(df.info())

# 3. Convert all categorical values to numeric using LabelEncoder
label_encoders = {}
for column in df.columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le  # Bewaren indien omgekeerde mapping later nodig is

# 4. Separate inputs and output
X = df.drop('class', axis=1)
y = df['class']

# 5. Normalize input data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)


  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB
None


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

# Split de data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Bouw het model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Outputlaag met 2 klassen

# Compileer het model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train het model
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

# Evalueer het model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

# Sla het model op
model.save('tic-tac-toe.model')


ImportError: Traceback (most recent call last):
  File "C:\Users\jdvdb\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Een initialisatieroutine van de dynamic link library (DLL-bestand) is mislukt.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the saved model
model = load_model('tic-tac-toe.model')

# Select a few random rows from the test dataset
num_samples = 5
indices = np.random.choice(len(X_test), size=num_samples, replace=False)
X_sample = X_test[indices]
y_sample = y_test.iloc[indices].values

# Make predictions
predictions = model.predict(X_sample)
predicted_classes = np.argmax(predictions, axis=1)

# Compare predictions with actual labels
for i in range(num_samples):
    print(f"Sample {i+1}: Predicted = {predicted_classes[i]}, Actual = {y_sample[i]}")


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [6]:
from tensorflow.keras.optimizers import Adam

# Bouw een verbeterd model
improved_model = Sequential()
improved_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
improved_model.add(Dense(64, activation='relu'))
improved_model.add(Dense(32, activation='relu'))
improved_model.add(Dense(2, activation='softmax'))

# Pas de learning rate aan via een custom Adam-optimizer
custom_optimizer = Adam(learning_rate=0.0005)

# Compileer het model met aangepaste optimizer
improved_model.compile(optimizer=custom_optimizer,
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

# Train met verhoogd aantal epochs
improved_model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

# Evalueer het verbeterde model
loss, accuracy = improved_model.evaluate(X_test, y_test)
print(f"Improved Model - Test Accuracy: {accuracy:.4f}, Loss: {loss:.4f}")


ImportError: Traceback (most recent call last):
  File "C:\Users\jdvdb\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Een initialisatieroutine van de dynamic link library (DLL-bestand) is mislukt.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# I tried a few things to improve the model.
# Adding more layers definitely helped — the model was able to learn more complex patterns.
# I also increased the number of epochs from 20 to 50, which gave the model more time to improve.
# That made a noticeable difference in both accuracy and loss.
# 
# I adjusted the learning rate to 0.0005 to make training a bit more stable.
# It didn’t change the final accuracy much, but it helped reduce noisy fluctuations in the loss.
#
# Overall, adding layers and training longer had the biggest impact.
# The learning rate tweak was useful, but not as critical.
